In [220]:
!pip install -q hopsworks[python]

In [221]:
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks


import warnings
warnings.filterwarnings("ignore")

# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

project = hopsworks.login()
fs = project.get_feature_store()

# Retrieve feature groups for Clash Royale
player_stats_fg = fs.get_feature_group(
    name='clash_royale_onehotencoding',
    version=5,
)
game_events_fg = fs.get_feature_group(
    name='clash_royale_dataset_onehotencoding',
    version=5,
)

df = player_stats_fg.read()




'''
# Retrieve features from feature groups
player_stats = player_stats_fg.select_all()
game_events = game_events_fg.select_all()

# Explicitly prefix column names to avoid ambiguity
player_stats = player_stats.with_column_renamed("player_name", "player_stats_player_name")
game_events = game_events.with_column_renamed("player_name", "game_events_player_name")

# Join the feature groups on the prefixed columns
selected_features = player_stats.join(
    game_events,
    on=["player_stats_player_name", "game_events_player_name"]
)

# Display the first 10 rows of the joined features
selected_features.show(10)


feature_view = fs.get_or_create_feature_view(
    name='clash_royale_fv',
    description="Clash Royale game prediction features",
    version=1,
    labels=['player_name'],
    #query=selected_features,
)
'''

2025-01-08 14:38:31,240 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-08 14:38:31,285 INFO: Initializing external client
2025-01-08 14:38:31,285 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 14:38:32,864 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164438
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.01s) 


'\n# Retrieve features from feature groups\nplayer_stats = player_stats_fg.select_all()\ngame_events = game_events_fg.select_all()\n\n# Explicitly prefix column names to avoid ambiguity\nplayer_stats = player_stats.with_column_renamed("player_name", "player_stats_player_name")\ngame_events = game_events.with_column_renamed("player_name", "game_events_player_name")\n\n# Join the feature groups on the prefixed columns\nselected_features = player_stats.join(\n    game_events,\n    on=["player_stats_player_name", "game_events_player_name"]\n)\n\n# Display the first 10 rows of the joined features\nselected_features.show(10)\n\n\nfeature_view = fs.get_or_create_feature_view(\n    name=\'clash_royale_fv\',\n    description="Clash Royale game prediction features",\n    version=1,\n    labels=[\'player_name\'],\n    #query=selected_features,\n)\n'

In [222]:
dfstats = player_stats_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.58s) 


In [223]:
dfdata = game_events_fg.read()
dfdata

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (113.23s) 


,player1,player2,deck1,deck2,result
0,#28JVJUJY9,#8RG9LP82C,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,#YCVURYC9V,#PLUCUGQV2,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,#CQCY920C,#C28QJCL,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0
3,#LUJ9QLP9L,#82YQC0800,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,#8UU9VPLPJ,#2JRU8Q8JR,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...
181874,#90LGPV2L,#R82Q0J0C9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
181875,#Y99C0GYJ,#PR0VCP89V,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
181876,#YU8L80VPG,#YQVJ0GCRU,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
181877,#UQ0QQRC,#90UYLRQQJ,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [262]:
dfdata

,player1,player2,deck1,deck2,result
0,#28JVJUJY9,#8RG9LP82C,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,#YCVURYC9V,#PLUCUGQV2,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,#CQCY920C,#C28QJCL,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0
3,#LUJ9QLP9L,#82YQC0800,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,#8UU9VPLPJ,#2JRU8Q8JR,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...
181874,#90LGPV2L,#R82Q0J0C9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
181875,#Y99C0GYJ,#PR0VCP89V,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
181876,#YU8L80VPG,#YQVJ0GCRU,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
181877,#UQ0QQRC,#90UYLRQQJ,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [263]:
df_1 = dfdata.iloc[:90000,:]
df_test = dfdata.iloc[91000:181000, :]


In [264]:
df_test['result']

91000     1
91001     0
91002     0
91003     1
91004     0
         ..
180995    0
180996    1
180997    0
180998    0
180999    1
Name: result, Length: 90000, dtype: object

In [265]:
# prompt: what are the different values for the column result

import pandas as pd

# Assuming dfdata is your DataFrame
# Replace with your actual DataFrame if it's named differently

# Check for unique values in the 'result' column
unique_results = dfdata['result'].unique()

# Print the unique values
unique_results

array(['1', '0'], dtype=object)

In [266]:
import pandas as pd

# Assuming the DataFrame is named df_1
# Create binary columns for deck1
deck1_columns = [f"deck1_{i}" for i in range(181)]
deck1_df = pd.DataFrame(df_1['deck1'].tolist(), columns=deck1_columns)

# Create binary columns for deck2
deck2_columns = [f"deck2_{i}" for i in range(181)]
deck2_df = pd.DataFrame(df_1['deck2'].tolist(), columns=deck2_columns)

# Combine all columns with the result column
result_column = df_1['result']
final_df = pd.concat([deck1_df, deck2_df, result_column], axis=1)

# Display or save the final DataFrame
final_df.head()  # To preview the DataFrame
# You can save the DataFrame to a CSV file if needed:


,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [267]:
final_df

,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
89996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
89997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89998,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [268]:
# Count non-zero elements in each row for deck1 and deck2
deck1_non_zero_count = (deck1_df != 0).sum(axis=1)
deck2_non_zero_count = (deck2_df != 0).sum(axis=1)

print(deck1_non_zero_count)

0        8
1        7
2        7
3        8
4        7
        ..
89995    6
89996    8
89997    7
89998    8
89999    8
Length: 90000, dtype: int64


In [231]:
'''
DOBBIAMO CREARE ANCHE I DATI OPPOSTI!
'''

'\nDOBBIAMO CREARE ANCHE I DATI OPPOSTI!\n'

In [232]:
# Create a new DataFrame with deck2 first, then deck1, and result column set to 0
#result_column_zero = pd.Series([0] * len(df_1), name="result")
#reversed_df = pd.concat([deck2_df, deck1_df, result_column_zero], axis=1)
#reversed_df

In [269]:
df_complete = pd.concat([final_df], ignore_index=True)
df_complete

,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
89996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
89997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89998,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [270]:
df_shuffled = df_complete.sample(frac=1, random_state=42).reset_index(drop=True)
final_df = df_shuffled.copy()
final_df

,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
89996,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
89997,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89998,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [271]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare data
X = final_df.iloc[:, :-1].values  # Features: all columns except the result
y = final_df.iloc[:, -1].values  # Target: the result column

In [272]:
y = y.astype(int)  # Convert directly if y is a Pandas Series or NumPy array
y

array([0, 0, 1, ..., 0, 1, 1])

In [273]:
# Split data into training and testing sets
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
#X_train = np.array([np.array(x) for x in X_train])
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='tanh')  # Output layer with tanh activation for -1 to 1 range
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



Epoch 1/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5571 - loss: 0.2623 - val_accuracy: 0.6025 - val_loss: 0.2366
Epoch 2/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6203 - loss: 0.2272 - val_accuracy: 0.5988 - val_loss: 0.2352
Epoch 3/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6362 - loss: 0.2197 - val_accuracy: 0.6016 - val_loss: 0.2391
Epoch 4/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6605 - loss: 0.2108 - val_accuracy: 0.5730 - val_loss: 0.2438
Epoch 5/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6864 - loss: 0.2013 - val_accuracy: 0.5845 - val_loss: 0.2475
Epoch 6/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7105 - loss: 0.1892 - val_accuracy: 0.5742 - val_loss: 0.2526
Epoch 7/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7357 - loss: 0.1791 - val_accuracy: 0.5709 - val_loss: 0.2623
Epoch 8/20
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7589 - loss: 0.1675 - 

In [274]:
y = (y == 1).astype(int)  # Map -1 to 0 and keep 1 as 1
y

array([0, 0, 1, ..., 0, 1, 1])

In [275]:
print("Unique classes in y after transformation:", set(y))


Unique classes in y after transformation: {0, 1}


In [276]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [277]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb_model.fit(X_train, y_train)

# Predict outcomes for the test set
y_pred = xgb_model.predict(X_test)

# Invert predictions if necessary
y_pred_corrected = 1 - y_pred  # Flip the predictions

# Evaluate the model with corrected predictions
xgb_accuracy_corrected = accuracy_score(y_test, y_pred_corrected)
xgb_report_corrected = classification_report(y_test, y_pred_corrected)

# Print results
print(f"XGBoost Test Accuracy (Corrected): {xgb_accuracy_corrected:.4f}")
print("Classification Report (Corrected Predictions):\n", xgb_report_corrected)


XGBoost Test Accuracy (Corrected): 0.3915
Classification Report (Corrected Predictions):
               precision    recall  f1-score   support

           0       0.48      0.12      0.19     10855
           1       0.38      0.80      0.51      7145

    accuracy                           0.39     18000
   macro avg       0.43      0.46      0.35     18000
weighted avg       0.44      0.39      0.32     18000



In [278]:
df_test

,player1,player2,deck1,deck2,result
91000,#YY0UYU8VQ,#PLGP92LC,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
91001,#P0PYRGVRQ,#9YRUGPRJ8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
91002,#2UJYP0U8,#8RPL00YR9,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
91003,#9CQ00QYR,#9RJ8L22RJ,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",1
91004,#YCPG0UQJC,#9Y20GPR02,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...
180995,#8PQ8JCV9R,#289Y2C08,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
180996,#JY80P0L,#2PC2U2GU,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
180997,#88CG908UU,#PGQ2PQJGV,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
180998,#98L90CV0R,#2PU9R0QRU,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [279]:
df_test['result']

91000     1
91001     0
91002     0
91003     1
91004     0
         ..
180995    0
180996    1
180997    0
180998    0
180999    1
Name: result, Length: 90000, dtype: object

In [280]:
# Creare le colonne binarie per deck1
deck1_test_df = pd.DataFrame(df_test['deck1'].tolist(), columns=deck1_columns).reset_index(drop=True)

# Creare le colonne binarie per deck2
deck2_test_df = pd.DataFrame(df_test['deck2'].tolist(), columns=deck2_columns).reset_index(drop=True)

# Combinare tutte le colonne con la colonna result
result_column_test = df_test['result'].reset_index(drop=True)
final_test_df = pd.concat([deck1_test_df, deck2_test_df, result_column_test], axis=1).reset_index(drop=True)

# Contare gli elementi non-zero per ogni riga in deck1 e deck2 (opzionale, se richiesto)
deck1_test_non_zero_count = (deck1_test_df != 0).sum(axis=1)
deck2_test_non_zero_count = (deck2_test_df != 0).sum(axis=1)
final_test_df 


,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck1_5,deck1_6,deck1_7,deck1_8,deck1_9,...,deck2_172,deck2_173,deck2_174,deck2_175,deck2_176,deck2_177,deck2_178,deck2_179,deck2_180,result
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
89996,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
89997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [281]:
X_test = final_test_df.iloc[:, :-1].values
y_test = final_test_df.iloc[:, -1].values

In [282]:
y_test =y_test.astype(int)

In [283]:
y_pred

array([0, 0, 0, ..., 0, 1, 1])

In [284]:
y_test

array([1, 0, 0, ..., 0, 0, 1])

In [285]:
# Predict outcomes for the test set
y_pred = xgb_model.predict(X_test)

# Invert predictions if necessary
y_pred_corrected = 1 - y_pred  # Flip the predictions

# Evaluate the model with corrected predictions
xgb_accuracy_corrected = accuracy_score(y_test, y_pred)
xgb_report_corrected = classification_report(y_test, y_pred)

# Print results
print(f"XGBoost Test Accuracy (Corrected): {xgb_accuracy_corrected:.4f}")
print("Classification Report (Corrected Predictions):\n", xgb_report_corrected)

XGBoost Test Accuracy (Corrected): 0.6099
Classification Report (Corrected Predictions):
               precision    recall  f1-score   support

           0       0.63      0.88      0.73     54395
           1       0.52      0.20      0.29     35605

    accuracy                           0.61     90000
   macro avg       0.57      0.54      0.51     90000
weighted avg       0.58      0.61      0.56     90000



In [286]:
# Predicting target values on the test set

# Calculating Mean Squared Error (MSE) using sklearn
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# Calculating R squared using sklearn
r2 = r2_score(y_test, y_pred)
print("R squared:", r2)

MSE: 0.39014444444444446
R squared: -0.6317006056549399


In [287]:
# Print 10 examples of y_test and y_pred_corrected
print("\nExamples of y_test and Corrected Predictions:")
for i in range(100):
    print(f"{y_test[i]} {y_pred[i]}")


Examples of y_test and Corrected Predictions:
1 0
0 0
0 0
1 0
0 0
0 0
0 0
1 0
1 1
0 0
0 0
0 0
0 0
0 0
0 1
0 0
1 0
0 0
1 1
1 0
1 0
0 1
0 0
0 0
0 0
0 0
0 0
0 1
0 0
1 0
0 0
0 0
1 1
1 0
1 0
0 1
0 0
1 0
0 0
0 0
0 1
1 0
1 1
0 0
1 1
0 0
0 0
0 1
0 0
0 1
0 0
0 1
0 0
1 0
0 0
1 1
1 1
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 1
1 0
1 0
0 0
0 0
1 0
1 0
1 0
0 0
0 0
0 0
0 0
0 0
1 0
1 0
1 1
0 0
1 0
0 0
1 0
0 0
0 0
0 0
1 0
1 0
0 0
1 1
1 1
0 0
0 0


In [288]:
# Creating a directory for the model artifacts if it doesn't exist
model_dir = "clash_royale_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)


In [289]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Creating input and output schemas using the 'Schema' class for features (X) and target variable (y)
input_schema = Schema(X_train)
output_schema = Schema(y_train)

# Creating a model schema using 'ModelSchema' with the input and output schemas
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# Converting the model schema to a dictionary representation
schema_dict = model_schema.to_dict()

In [290]:
xgb_model.save_model(model_dir + "/model.json")

In [291]:
res_dict = { 
        "MSE": str(mse),
        "R squared": str(r2),
    }

In [292]:
import numpy as np

# Select a random sample from X_test
input_example = X_test[np.random.choice(X_test.shape[0], 1, replace=False)]


In [293]:
mr = project.get_model_registry()

# Creating a Python model in the model registry named 'air_quality_xgboost_model'

aq_model = mr.python.create_model(
    name="clash_royale_xgboost_model", 
    metrics= res_dict,
    model_schema=model_schema,
    input_example=input_example, 
    description="Win Probability rate",
)

# Saving the model artifacts to the 'air_quality_model' directory in the model registry
aq_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/606520 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1088 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/226 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1164438/models/clash_royale_xgboost_model/6


Model(name: 'clash_royale_xgboost_model', version: 6)

In [258]:
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [259]:
selected_features = game_events_fg.select(['deck1','deck2','result'])

In [260]:
selected_features

In [261]:
feature_view = fs.get_or_create_feature_view(
    name='clashroyale_fv',
    description="deck features",
    version=3,
    labels=['result'],
    query=selected_features,
)

ValueError: Cannot get back the feature view because the query defined is no longer valid. Some feature groups used in the query may have been deleted. You can clean up this feature view on the UI or `FeatureView.clean`.

In [ ]:
#feature_view.read()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.13s) 


In [ ]:
X_train

,deck1,deck2
0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
620,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
621,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
622,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
623,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
import pandas as pd

# Assuming x_train is already loaded into memory, we'll convert the deck1 and deck2 columns into arrays of numbers.
# Simulating an example since the actual x_train content is not directly available.

# Example data simulation


# Transform columns into arrays of numbers
X_train["deck1"] = X_train["deck1"].apply(lambda x: list(x))
X_train["deck2"] = X_train["deck2"].apply(lambda x: list(x))

X_train


,deck1,deck2
0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
620,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
621,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
622,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
623,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
X_test = X_test.dropna()

X_train = X_train.dropna()




In [ ]:
y_train = y_train.dropna()

y_test = y_test.dropna()

In [ ]:
# Example of flattening columns
deck_flat = pd.DataFrame(data=list(X_train['deck1']), index=X_train.index)
deck2_flat = pd.DataFrame(data=list(X_train['deck2']), index=X_train.index)

# Merge flattened columns into the dataset
#X_train = pd.concat([X_train.drop(['deck1', 'deck2'], axis=1), deck_flat, deck2_flat], axis=1)
#X_test = pd.concat([X_test.drop(['deck1', 'deck2'], axis=1),
                #    pd.DataFrame(data=list(X_test['deck1']), index=X_test.index),
                 #   pd.DataFrame(data=list(X_test['deck2']), index=X_test.index)], axis=1)


In [ ]:
print(X_test.dtypes)


deck1    object
deck2    object
dtype: object


In [ ]:
X_train


,deck1,deck2
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
620,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
621,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
622,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
623,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
y_train

,result
1,1
2,1
3,1
4,1
5,1
...,...
620,1
621,1
622,1
623,1


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Creare il modello
model = Sequential([
    Input(shape=(624,)),  # Dimensione del vettore d'ingresso
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Per classificazione binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Addestrare il modello
model.fit(X_train, y_train, epochs=10, batch_size=32)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
# Drop unnecessary columns
#train_features = X_train.drop(['date', 'player_id'], axis=1)
#test_features = X_test.drop(['date', 'player_id'], axis=1)

# Create and train the XGBoost Regressor
#xgb_regressor = XGBRegressor()
xgb_regressor = XGBRegressor(enable_categorical=True)
xgb_regressor.fit(X_train, y_train)

# Predict game outcomes on the test set
y_pred = xgb_regressor.predict(X_test)

# Calculate and print metrics
mse = mean_squared_error(y_test.iloc[:, 0], y_pred)
print("MSE:", mse)

r2 = r2_score(y_test.iloc[:, 0], y_pred)
print("R squared:", r2)

# Save predictions to DataFrame
df = y_test
df['predicted_game_outcome'] = y_pred

# Create directory for model artifacts
model_dir = "clash_royale_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
images_dir = model_dir + "/images"
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

# Visualize and save feature importance
plot_importance(xgb_regressor, max_num_features=4)
feature_importance_path = images_dir + "/feature_importance.png"
plt.savefig(feature_importance_path)
plt.show()

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define input and output schemas
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
schema_dict = model_schema.to_dict()

# Save the XGBoost model
xgb_regressor.save_model(model_dir + "/model.json")
mr = project.get_model_registry()

# Register the model in the Hopsworks model registry
cr_model = mr.python.create_model(
    name="clash_royale_xgboost_model",
    metrics={"mse": mse, "r2": r2},
    model_schema=model_schema,
    input_example=X_test.sample().values,
    description="Clash Royale game outcome predictor",
)

# Save the model artifacts to the registry
cr_model.save(model_dir)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:deck1: object, deck2: object

In [ ]:
import pandas as pd
import numpy as np

# Example DataFrame
X_train = pd.DataFrame({
    "deck1": [[0, 1, 0, 1, 0], [0, 0, 1, 0, 1], [1, 0, 0, 0, 0]],
    "deck2": [[1, 0, 0, 1, 0], [0, 0, 1, 0, 0], [1, 1, 0, 0, 1]],
    "other_feature": [5, 10, 15]
})

# Expand `deck1` and `deck2` arrays into multiple numerical columns
deck1_expanded = pd.DataFrame(X_train["deck1"].to_list(), columns=[f"deck1_{i}" for i in range(len(X_train["deck1"].iloc[0]))])
deck2_expanded = pd.DataFrame(X_train["deck2"].to_list(), columns=[f"deck2_{i}" for i in range(len(X_train["deck2"].iloc[0]))])

# Combine expanded decks with other features
X_train_transformed = pd.concat([deck1_expanded, deck2_expanded, X_train.drop(columns=["deck1", "deck2"])], axis=1)

X_train_transformed.head()


,deck1_0,deck1_1,deck1_2,deck1_3,deck1_4,deck2_0,deck2_1,deck2_2,deck2_3,deck2_4,other_feature
0,0,1,0,1,0,1,0,0,1,0,5
1,0,0,1,0,1,0,0,1,0,0,10
2,1,0,0,0,0,1,1,0,0,1,15
